# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [3]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

df = pd.read_csv('ToothGrowth.csv')
df

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5
5,10.0,VC,0.5
6,11.2,VC,0.5
7,11.2,VC,0.5
8,5.2,VC,0.5
9,7.0,VC,0.5


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [4]:
formula = 'len ~ C(supp) + dose'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df           F        PR(>F)
C(supp)    205.350000   1.0   11.446768  1.300662e-03
dose      2224.304298   1.0  123.988774  6.313519e-16
Residual  1022.555036  57.0         NaN           NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

In [ ]:
# Both supp and dose appear to have a significant effect on length, with dose being most important

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [6]:
df_OJ = df.loc[df['supp'] == 'OJ']

df_VC = df.loc[df['supp'] == 'VC']

Now run a t-test between these two groups and print the associated two-sided p-value: 

In [8]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
import flatiron_stats as fl

fl.p_value_welch_ttest(df_OJ.len, df_VC.len, two_sided=True)

0.06063450788093383

## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [9]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
# Compare the p-value to that of the t-test above. 
# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)

formula = 'len ~ C(supp)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df         F    PR(>F)
C(supp)    205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [7]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [10]:
from itertools import combinations

In [11]:
groups = []
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    groups.append(group_name)

In [30]:
combos = list(combinations(groups, 2))
combos[0]

(('OJ', 0.5), ('OJ', 1.0))

In [34]:
for combo in combos:
    # Each iteration will involve two supps and two doses
    supp1 = combo[0][0]
    dose1 = combo[0][1]
    supp2 = combo[1][0]
    dose2 = combo[1][1]
    
    # Looking to compare a sample1 and sample2 each time
    sample1 = df.loc[(df.supp == supp1) & (df.dose == dose1)]['len']
    sample2 = df.loc[(df.supp == supp2) & (df.dose == dose2)]['len']
    
    # Generate p_value for the combo, then print combination and p_value
    p_value = fl.p_value_welch_ttest(sample1, sample2, two_sided=False)
    print(combo, p_value)

(('OJ', 0.5), ('OJ', 1.0)) 4.3924595275801614e-05
(('OJ', 0.5), ('OJ', 2.0)) 6.618919389245193e-07
(('OJ', 0.5), ('VC', 0.5)) 0.003179303382048415
(('OJ', 0.5), ('VC', 1.0)) 0.02300516628818783
(('OJ', 0.5), ('VC', 2.0)) 3.5981267619833446e-06
(('OJ', 1.0), ('OJ', 2.0)) 0.019597571023121985
(('OJ', 1.0), ('VC', 0.5)) 1.827603357540397e-08
(('OJ', 1.0), ('VC', 1.0)) 0.0005191879361500229
(('OJ', 1.0), ('VC', 2.0)) 0.048263061691335096
(('OJ', 2.0), ('VC', 0.5)) 6.810774166865485e-12
(('OJ', 2.0), ('VC', 1.0)) 1.1805371002981957e-07
(('OJ', 2.0), ('VC', 2.0)) 0.4819257943616878
(('VC', 0.5), ('VC', 1.0)) 3.4055088515838605e-07
(('VC', 0.5), ('VC', 2.0)) 2.3407887073112477e-08
(('VC', 1.0), ('VC', 2.0)) 4.577801528327097e-05


In [ ]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)

## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.